<a href="https://colab.research.google.com/github/Sooryakiran/CS6886_SysDL/blob/master/assignment_3/Submission/Step%204/step_4_gemm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implimentation without nn.Conv2d and nn.Linear

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import time
import copy
from IPython.display import HTML, display
from torchsummary import summary

### Define useful classes and functions

In [0]:
class Dense(nn.Module):
    """
    This is the custom implimentation of nn.Linear

    """
    def __init__(self, in_channels, out_channels):
        """
        The class constructor.

        @param in_channels  : Number of neurons in the previous layer.
        @param out_channels : Number of neurons in the current layer.

        """

        super(Dense, self).__init__()
        self.weight = torch.nn.Parameter(data = torch.Tensor(out_channels, in_channels), requires_grad = True)
        self.bias   = torch.nn.Parameter(data = torch.Tensor(out_channels), requires_grad = True)
    
    def forward(self, x):
        """
        Computes the activations of the current layer.

        @param x : Input to the current layer.
        @return  : Outputs of the current layer

        """

        return self.bias + x @ torch.transpose(self.weight, 1, 0)


def isnan(x):
    """
    Check if there is any NaNs in a given tensor.

    @param x : Input tensor
    @return  : Boolean tensor

    """

    return torch.isnan(torch.sum(x))


class Conv2d(nn.Module):
    """
    This is the custom implimentation of nn.Conv2d

    NOTE : This is not differentiable.

    """
    def __init__(self, in_channels, out_channels, kernel_size):
        """
        The class constructor.

        @param in_channels  : Number of channels in the inputs.
        @param out_channels : Number of conv. filters.
        @param kernel_size  : Size of the conv. kernels.

        """
        
        super(Conv2d, self).__init__()
        self.out_channels = out_channels
        self.in_channels  = in_channels

        if isinstance(kernel_size, int):
            kernel_size = [kernel_size, kernel_size]
        if len(kernel_size) == 1:
            kernel_size = [kernel_size[0], kernel_size[0]]

        self.kernel_size = kernel_size
        self.weight = torch.nn.Parameter(data = torch.Tensor(out_channels,\
                                                            in_channels,\
                                                            kernel_size[0],\
                                                            kernel_size[1]), requires_grad=True)
        self.bias   = torch.nn.Parameter(data = torch.Tensor(out_channels), requires_grad = True)

    def forward(self, x):
        """
        Computes the activations of a conv layer by converting to GEMM using teopltiz matrix.

        @param x : Inputs.
        @return  : Current layer activations.

        NOTE     : torch.unfold(...) used to create the teoplitz matrix is not differentiable.
                   Hence this can only be used for inference.

        """

        weight_     = self.weight.view((self.out_channels, -1))
        num_batches = x.size()[0]
        in_channels = self.in_channels
        k           = self.kernel_size

        teoplitz_full_list = []
        big_unfolding = x.unfold(2, k[0], 1).unfold(3, k[1], 1).contiguous().view(num_batches, in_channels, -1, k[0]*k[1]).permute(0, 1, 3, 2)
        teoplitz_full = big_unfolding.contiguous().view(num_batches, in_channels*k[0]*k[1], -1).permute(1, 0, 2).contiguous().view(in_channels*k[0]*k[1], -1)
        out_size      = np.array([x.size()[-2], x.size()[-1]]) - np.array(k) + np.array([1, 1])
        
        raw  = weight_ @ teoplitz_full
        raw  = raw.view(self.out_channels, num_batches, out_size[0], out_size[1]).permute(1, 0, 2, 3)
        bias = self.bias.unsqueeze(dim = -1).unsqueeze(dim = -1).unsqueeze(dim = 0).repeat(num_batches, 1, out_size[0], out_size[1])

        return (raw + bias).contiguous()
        
class ConvNetMine(nn.Module):
    """
    The network is defined as a torch module.

    """

    def __init__(self,\
                 conv_layer_1_filter_size,\
                 conv_layer_2_filter_size,\
                 conv_layer_1_channel_size,\
                 conv_layer_2_channel_size,\
                 layer_3_size):
        
        """
        The class constructor.

        """

        super(ConvNetMine, self).__init__()

        self.conv_1 = Conv2d(in_channels = 1, out_channels = conv_layer_1_channel_size,  kernel_size = conv_layer_1_filter_size)
        self.conv_2 = Conv2d(in_channels = conv_layer_1_channel_size, out_channels = conv_layer_2_channel_size, kernel_size = conv_layer_2_filter_size)
        
        size = 28
        size = size - conv_layer_1_filter_size + 1
        size = size - conv_layer_2_filter_size + 1
        self.fc_1_input_size = conv_layer_2_channel_size*size*size
        self.fc_1   = Dense(self.fc_1_input_size, layer_3_size)
        self.fc_2   = Dense(layer_3_size, 10)

    def forward(self, x):
        """
        The forward pass of the network.

        Layer 1: Convolution 3x3 16 channels with relu
        Layer 2: Convolution 3x3 16 channels with relu
        Layer 3: Fully connected 100 neurons with relu
        Layer 4: Output fully connected layer 10 neurons without activation 

        @param x : Input tensor of size (BATCH_SIZE, 1, 28, 28)
        @return  : Network output tensor of size (BATCH_SIZE, 10)

        """
        x = F.relu(self.conv_1(x))
        x = F.relu(self.conv_2(x))
        x = x.view(-1,  self.fc_1_input_size)
        x = F.relu(self.fc_1(x))
        x = self.fc_2(x)
        return x


### Other objects and functions

In [0]:
BATCH_SIZE = 32
INITIALIZAION = torch.nn.init.kaiming_uniform_
LEARNING_RATE = 0.01
MOMENTUM = 0.9
LOG_DIR = "drive/My Drive/SysDL/Assg 3/GEMM"

In [0]:
class NormalizeTransform:
    """
    A torch transform class that normalises the dataset

    """
    
    def __init__(self, mean = 0.5, var = 0.5):
        """
        The class constructor

        @param mean : mean of the output dataset
        @param var  : variance of the output datset

        """

        self.mean = mean
        self.var  = var
    
    def __call__(self, x):
        """
        This function is called when the object is called.

        @param x : input datset
        @returns : normalized dataset

        """

        return (x-self.mean)/(self.var)


def imshow(image):
    """
    A function to print images of size 28x28 from the dataset

    @param image: input torch tensor representing the image

    """

    npimg = image.numpy()
    plt.imshow(np.resize(npimg, (28, 28)), cmap = 'gray')
    plt.show()

def progress(value, max=100):
    """
    Progress bar

    """
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value = value, max = max))

class ConvNet(nn.Module):
    """
    The network is defined as a torch module.

    """

    def __init__(self,\
                 conv_layer_1_filter_size,\
                 conv_layer_2_filter_size,\
                 conv_layer_1_channel_size,\
                 conv_layer_2_channel_size,\
                 layer_3_size):
        
        """
        The class constructor.

        """

        super(ConvNet, self).__init__()

        self.conv_1 = nn.Conv2d(in_channels = 1, out_channels = conv_layer_1_channel_size,  kernel_size = conv_layer_1_filter_size)
        self.conv_2 = nn.Conv2d(in_channels = conv_layer_1_channel_size, out_channels = conv_layer_2_channel_size, kernel_size = conv_layer_2_filter_size)
        
        size = 28
        size = size - conv_layer_1_filter_size + 1
        size = size - conv_layer_2_filter_size + 1
        self.fc_1_input_size = conv_layer_2_channel_size*size*size
        self.fc_1   = nn.Linear(self.fc_1_input_size, layer_3_size)
        self.fc_2   = nn.Linear(layer_3_size, 10)

    def forward(self, x):
        """
        The forward pass of the network.

        Layer 1: Convolution 3x3 16 channels with relu
        Layer 2: Convolution 3x3 16 channels with relu
        Layer 3: Fully connected 100 neurons with relu
        Layer 4: Output fully connected layer 10 neurons without activation 

        @param x : Input tensor of size (BATCH_SIZE, 1, 28, 28)
        @return  : Network output tensor of size (BATCH_SIZE, 10)

        """
        x = F.relu(self.conv_1(x))
        x = F.relu(self.conv_2(x))
        x = x.view(-1,  self.fc_1_input_size)
        x = F.relu(self.fc_1(x))
        x = self.fc_2(x)
        return x


class initParams:
    """
    An object to initialize the network parameters throgh the method specified

    """

    def __init__(self, method = torch.nn.init.xavier_uniform_):
        """
        The class constructor.

        @param method: the torch.nn.init function to be used to initialize the
                       network parameters

        """

        self.method = method
    
    def __call__(self, layer):
        """
        A function to initialize the network parameters. The weights are initialized
        with given method (default = Xavier uniform) initialization. The biases are 
        initialized with zeros.

        Usage:
            net.apply(init_params)
            where net is an nn.Module or nn.Seqential
        
        @param layer : each layers of the nn.Module/nn.Sequential

        """

        if type(layer) == nn.Linear:
            self.method(layer.weight)
            layer.bias.data.fill_(0.0)

        elif type(layer) == nn.Conv2d:
            self.method(layer.weight)
            layer.bias.data.fill_(0.0)


def accuracy(pred, target):
    """
    A function to calculate accuracy of a prediction.

    @param pred   : Network predictions tensor of size (BATCH_SIZE, num outputs)
    @param target : Target labels of size (BATCh_SIZE)

    Note: @param target is NOT onehot encoded.

    @return       : accuracy of the prediction.

    """

    preds    = torch.argmax(pred, dim = -1).detach().cpu().numpy()
    target   = target.detach().cpu().numpy()
    corrects = np.mean(np.asarray([preds[i] == target[i] for i in range(preds.shape[0])]))
    return corrects


def test(network, test_loader, criterion, accuracy_fn, device = "cpu"):
    """
    This function evaluates a model on the test dataset.

    @param network     : Neural network model
    @param test_loader : The dataloader object for the test dataset
    @param criterion   : The loss function
    @param accuracy_fn : The accuracy function

    @return test_loss, test_accuracy 

    """

    loss     = 0.0
    accuracy = 0.0
    batches  = 0

    for i, data in enumerate(test_loader):
        inputs, labels = data
        if device == "cuda":
            inputs = inputs.to("cuda")
            labels = labels.to("cuda")

        outputs        = network(inputs)

        loss     += criterion(outputs, labels)
        accuracy += accuracy_fn(outputs, labels)
        batches   = i + 1
    
    return (loss/batches).item(), (accuracy/batches).item()

def train_and_evaluate(network, criterion, optimizer, train_dataloader, test_dataloader, train_till = 90, verbose = True):
    """
    A function which trains the given network till the training accuracy reaches
    the specified value, on the given dataset, using the specified loss function
    and using the given optimzer.

    @param network          : Torch nn.Module or nn.Sequential network object
    @param criterion        : The loss function
    @param optimizer        : The optimizer object
    @param train_dataloader : The dataloader object of the training dataset
    @param test_dataloader  : The dataloader object of the test dataset
    @param train_till       : The training accuracy till which the network has 
                              to be trained
    @return t_acc           : Test accuracy on training till training accuracy 
                              becomes the specified value
    @return training_time   : Time required to reach the specified training 
                              accuracy

    """
    done = False

    start_time = time.time()
    while not done:

        running_loss     = 0.0
        running_accuracy = 0.0

        for i, data in enumerate(train_dataloader):
            if not done:
                inputs, labels = data
                optimizer.zero_grad()

                outputs = net(inputs)
                loss    = criterion(outputs, labels)

                loss.backward()
                optimizer.step()

                running_loss     += loss.item()
                running_accuracy += accuracy(outputs, labels)

                PRINT_EVERY = 256
                if i % PRINT_EVERY == PRINT_EVERY - 1:    
                    if verbose:
                        
                        
                        t_loss, t_acc = test(network, test_dataloader, criterion, accuracy)
                        print('[BATCH: %5d]\t Loss: %.3f\t Accuracy: %.3f %%\t Test Loss: %f\t Test Accuracy: %f %%' %
                            (i + 1, running_loss / PRINT_EVERY, running_accuracy*100 / PRINT_EVERY, t_loss, t_acc*100))

                    done             = running_accuracy*100/PRINT_EVERY > train_till
                    running_accuracy = 0.0
                    running_loss     = 0.0

    return t_acc, time.time() - start_time

def log(model_id, t_acc, log_dir = LOG_DIR):
    """
    A logging function

    """

    fp = open(log_dir + "logs.log", 'a')
    logs = "[MODEL ID: " + str(model_id) + "] " + "Test Accuracy: " + str(t_acc) +"\n"
    fp.write(logs)
    fp.close()

def train(network, criterion, optimizer, epochs, train_dataloader):
    """
    A handy function for training

    @param network              : The network nn.Module model for training
    @param criterion            : The loss function
    @param optimizer            : The optimizer
    @param epochs               : Number of epochs to train
    @param train_dataloader     : The train dataloader
    @return trained network

    """

    for epoch in range(epochs):
        total = 60000
        print("Epoch : %d" %(epoch + 1))
        out   = display(progress(0, 60000), display_id=True)

        running_loss     = 0.0
        running_accuracy = 0.0

        
        
        for i, data in enumerate(train_dataloader):
            inputs, labels = data
            optimizer.zero_grad()

            outputs = network(inputs)
            loss    = criterion(outputs, labels)

            

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            running_accuracy += accuracy(outputs, labels)
            PRINT_EVERY = 256

            if i % PRINT_EVERY == PRINT_EVERY - 1:
                batch_size       = labels.numpy().shape[0]
                running_accuracy = 0.0
                running_loss     = 0.0
                out.update(progress(i*batch_size, 60000))
        out.update(progress(1, 1))
        _, new_acc = test(network, test_dataloader, criterion, accuracy)
    return network


### Setup dataloaders

In [0]:
transform = transforms.Compose([transforms.ToTensor(), NormalizeTransform()])

train_dataset = torchvision.datasets.FashionMNIST(train = True,
                                  root = '.',
                                  download = True,
                                  transform = transform)

test_dataset = torchvision.datasets.FashionMNIST(train = False,
                                  root = '.',
                                  download = True, 
                                  transform = transform)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers = 0)
test_dataloader  = torch.utils.data.DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle = False, num_workers = 0)

Extracting ./FashionMNIST/raw/train-images-idx3-ubyte.gz to ./FashionMNIST/raw



Extracting ./FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./FashionMNIST/raw


Extracting ./FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./FashionMNIST/raw


Extracting ./FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./FashionMNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


### Train the choosen model or download the parameters

In [0]:
!rm -r torch_model.pth
!wget https://github.com/Sooryakiran/CS6886_SysDL/raw/master/assignment_3/Step%204/torch_model.pth
net = ConvNet(3, 3, 1, 2, 98)
net = torch.load("torch_model.pth")

--2020-05-09 08:43:44--  https://github.com/Sooryakiran/CS6886_SysDL/raw/master/assignment_3/Step%204/torch_model.pth
Resolving github.com (github.com)... 13.229.188.59
Connecting to github.com (github.com)|13.229.188.59|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Sooryakiran/CS6886_SysDL/master/assignment_3/Step%204/torch_model.pth [following]
--2020-05-09 08:43:45--  https://raw.githubusercontent.com/Sooryakiran/CS6886_SysDL/master/assignment_3/Step%204/torch_model.pth
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 467995 (457K) [application/octet-stream]
Saving to: ‘torch_model.pth’

torch_model.pth     100%[===================>] 457.03K  --.-KB/s    in 0.02s   

2020-05-09 08:43:45 (25.0 MB

### Or train again

In [0]:
net         = ConvNet(3, 3, 1, 2, 98)
init_params = initParams(method = INITIALIZAION)
net.apply(init_params)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = LEARNING_RATE, momentum = MOMENTUM)

test_acc, time_ = train_and_evaluate(network = net,\
                                    criterion=criterion,\
                                    optimizer=optimizer,\
                                    train_dataloader = train_dataloader,\
                                    test_dataloader = test_dataloader,\
                                    train_till=90)

### Create our ConvNet made without nn.Conv2d and nn.Linear

In [0]:
my_net = ConvNetMine(3, 3, 1, 2, 98)
parameters = net.state_dict()
my_net.load_state_dict(parameters)

criterion = nn.CrossEntropyLoss()

In [0]:
summary(my_net, (1, 28, 28))
summary(net, (1, 28, 28))

### Time their network

In [0]:
start_time = time.time()
test(net, test_dataloader, criterion, accuracy)
t = time.time() - start_time
print("Time :", t)

Time : 1.2817320823669434


### Time our network

In [0]:
start_time = time.time()
test(my_net, test_dataloader, criterion, accuracy)
t_2 = time.time() - start_time
print("Time:", t_2)

Time: 2.496305465698242


In [0]:
print("Increase :", 100*(t_2 - t)/t, "%")

Increase : 94.76031692117559 %


### CUDA

In [0]:
net2 = net.to("cuda")
my_net2 = my_net.to("cuda")

In [0]:
start_time = time.time()
test(net2, test_dataloader, criterion, accuracy, device = "cuda")
t = time.time() - start_time
print("Time :", t)

Time : 1.0231051445007324


In [0]:
start_time = time.time()
test(my_net2, test_dataloader, criterion, accuracy, device = "cuda")
t = time.time() - start_time
print("Time :", t)

Time : 1.2119534015655518
